In [1]:
import os

In [2]:
%pwd

'/Users/inamgurung/Desktop/Projects/AWS/Kidney-Disease-Classification/Kidney-Disease-Classification-Deep-Learning-Project/research'

In [3]:
os.chdir("../") # change directory to project root folder


In [4]:
%pwd

'/Users/inamgurung/Desktop/Projects/AWS/Kidney-Disease-Classification/Kidney-Disease-Classification-Deep-Learning-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import zipfile
import kagglehub
from cnnClassifier.utils.common import get_size
from cnnClassifier import logger

/Users/inamgurung/miniconda/envs/kidney/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [9]:
import shutil
import random

In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def _limit_files_per_class(self, base_dir: Path, class_name: str, max_files: int = 100):
        class_dir = Path(base_dir) / class_name

        files = [f for f in class_dir.iterdir() if f.is_file()]

        if len(files) <= max_files:
            return  # nothing to trim

        random.shuffle(files)
        files_to_remove = files[max_files:]

        for f in files_to_remove:
            f.unlink()


    def download_file(self):
        '''
        Fetch data from the url
        '''
        try: 
            dataset_url = self.config.source_URL
            unzip_path = self.config.unzip_dir
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from source URL {dataset_url} to {unzip_path}")

            # Code to download data from self.config.source_URL
            file_id = dataset_url.split("/")[-2] + "/" + dataset_url.split("/")[-1].split("?")[0]
            cache_path = Path(kagglehub.dataset_download(file_id))

            # Copy all files from cache to your project directory
            #shutil.copytree(cache_path, unzip_path, dirs_exist_ok=True)

            #LIMIT DATASET SIZE for speed during experimentation
            self._limit_files_per_class(unzip_path, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Cyst", max_files=100)
            self._limit_files_per_class(unzip_path, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Tumor", max_files=100)
            self._limit_files_per_class(unzip_path, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Normal", max_files=100)
            self._limit_files_per_class(unzip_path, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Cyst", max_files=100)
            self._limit_files_per_class(unzip_path, "CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/Stone", max_files=100)
            
            logger.info(f"Data downloaded to {self.config.local_data_file} into {unzip_path}")
        except Exception as e:
            logger.error(f"Error occurred while downloading data: {e}")
            raise e
    

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
except Exception as e:
    raise e


[2026-01-19 15:44:47,667 - INFO - yaml file: config/config.yaml loaded successfully]
[2026-01-19 15:44:47,681 - INFO - yaml file: params.yaml loaded successfully]
[2026-01-19 15:44:47,683 - INFO - Created directory at: artifacts]
[2026-01-19 15:44:47,684 - INFO - Created directory at: artifacts/data_ingestion]
[2026-01-19 15:44:47,685 - INFO - Downloading data from source URL https://www.kaggle.com/datasets/nazmul0087/ct-kidney-dataset-normal-cyst-tumor-and-stone?resource=download to artifacts/data_ingestion]
[2026-01-19 15:44:49,763 - INFO - Data downloaded to artifacts/data_ingestion/data.zip into artifacts/data_ingestion]
